## Extracción de datos

In [1]:
import pandas as pd 

In [2]:
import warnings
warnings.filterwarnings('ignore' )

In [3]:
url = r'C:/Users/agc13/Documents/Bootcamp/4-Semana/W5-ETL_project/Data/players.csv'

In [4]:
players = pd.read_csv(r'C:/Users/agc13/Documents/Bootcamp/4-Semana/W5-ETL_project/Data/players.csv', sep = ';', encoding = 'latin1')

In [5]:
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver

In [6]:
#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')

In [7]:
PATH = r'C:\Users\agc13\Documents\Bootcamp\4-Semana\4.3-lab-web-scraping-selenium\your-code\Driver\chromedriver.exe'

In [8]:
driver=webdriver.Chrome(PATH, options=opciones)

driver.get('https://state.1keydata.com/nba-teams-by-state.php')

In [9]:
nombre=driver.find_element_by_xpath('/html/body/center/table[2]/tbody/tr[1]/td/center/table/tbody/tr/td[1]/center/table')


### Transformación de datos

In [10]:
tablaraw = nombre.text
tablaraw

'State Name\nNBA Team(s)\nArizona Phoenix Suns\nCalifornia Golden State Warriors\nLos Angeles Clippers\nLos Angeles Lakers\nSacramento Kings\nColorado Denver Nuggets\nFlorida Miami Heat\nOrlando Magic\nGeorgia Atlanta Hawks\nIllinois Chicago Bulls\nIndiana Indiana Pacers\nLouisiana New Orleans Pelicans\nMassachusetts Boston Celtics\nMichigan Detroit Pistons\nMinnesota Minnesota Timberwolves\nNew York Brooklyn Nets\nNew York Knicks\nNorth Carolina Charlotte Hornets\nOhio Cleveland Cavaliers\nOklahoma Oklahoma City Thunder\nOregon Portland Trail Blazers\nPennsylvania Philadelphia 76ers\nTennessee Memphis Grizzlies\nTexas Dallas Mavericks\nHouston Rockets\nSan Antonio Spurs\nUtah Utah Jazz\nWisconsin Milwaukee Bucks\nWashington DC* Washington Wizards'

In [11]:
tablaraw = tablaraw.split("\n")


In [12]:
col = tablaraw[:2]

In [13]:
col

['State Name', 'NBA Team(s)']

In [14]:
filas=nombre.find_elements_by_tag_name('tr')

In [15]:
datos=[]


for f in filas:
    
    elementos=f.find_elements_by_tag_name('td')
    
    tmp=[]
    
    for elem in elementos:
        
        tmp.append(elem.text)
        
    datos.append(tmp)
        
datos

[['State Name', 'NBA Team(s)'],
 ['Arizona', 'Phoenix Suns'],
 ['California',
  'Golden State Warriors\nLos Angeles Clippers\nLos Angeles Lakers\nSacramento Kings'],
 ['Colorado', 'Denver Nuggets'],
 ['Florida', 'Miami Heat\nOrlando Magic'],
 ['Georgia', 'Atlanta Hawks'],
 ['Illinois', 'Chicago Bulls'],
 ['Indiana', 'Indiana Pacers'],
 ['Louisiana', 'New Orleans Pelicans'],
 ['Massachusetts', 'Boston Celtics'],
 ['Michigan', 'Detroit Pistons'],
 ['Minnesota', 'Minnesota Timberwolves'],
 ['New York', 'Brooklyn Nets\nNew York Knicks'],
 ['North Carolina', 'Charlotte Hornets'],
 ['Ohio', 'Cleveland Cavaliers'],
 ['Oklahoma', 'Oklahoma City Thunder'],
 ['Oregon', 'Portland Trail Blazers'],
 ['Pennsylvania', 'Philadelphia 76ers'],
 ['Tennessee', 'Memphis Grizzlies'],
 ['Texas', 'Dallas Mavericks\nHouston Rockets\nSan Antonio Spurs'],
 ['Utah', 'Utah Jazz'],
 ['Wisconsin', 'Milwaukee Bucks'],
 ['Washington DC*', 'Washington Wizards']]

In [16]:
datos=[[e.text for e in f.find_elements_by_tag_name('td')] for f in filas]

In [17]:
df = pd.DataFrame(datos[1:], columns=datos[0])

In [18]:
import numpy as np

In [19]:
df.rename(columns = {'State Name':'State_Name', r'NBA Team(s)':'NBA_Teams'}, inplace = True)

In [20]:
df = df.assign(NBA_Teams=df.NBA_Teams.str.split('\n')).explode('NBA_Teams')


In [21]:
players = pd.read_csv(r'C:/Users/agc13/Documents/Bootcamp/4-Semana/W5-ETL_project/Data/players.csv', sep = ';', encoding = 'latin1')

In [22]:
players.Tm.unique()

array(['POR', 'DEN', 'ORL', 'DET', 'WSB', 'DAL', 'TOT', 'MIL', 'NJN',
       'PHI', 'SAS', 'CHI', 'GSW', 'MIA', 'CLE', 'UTA', 'LAC', 'SEA',
       'ATL', 'PHO', 'BOS', 'CHH', 'SAC', 'MIN', 'Tm', 'LAL', 'HOU',
       'NYK', 'IND', 'VAN', 'TOR', 'WAS', 'MEM', 'NOH', 'CHA', 'NOK',
       'OKC', 'BRK', 'NOP', 'CHO'], dtype=object)

### Unificación de equipos que han cambiado de nombre

In [23]:
df.NBA_Teams.replace({'Phoenix Suns': 'PHO', 'Golden State Warriors': 'GSW', 'Los Angeles Clippers': 'LAC','Los Angeles Lakers':'LAL', 'Sacramento Kings':'SAC','Denver Nuggets': 'DEN','Miami Heat':'MIA', 'Orlando Magic':'ORL','Atlanta Hawks':'ATL','Chicago Bulls':'CHI', 'Indiana Pacers':'IND','New Orleans Pelicans':'NOP', 'Boston Celtics':'BOS','Detroit Pistons':'DET','Minnesota Timberwolves':'MIN','Brooklyn Nets':'BRK','New York Knicks':'NYK','Charlotte Hornets':'CHA', 'Cleveland Cavaliers':'CLE','Oklahoma City Thunder':'OKC','Portland Trail Blazers':'POR','Philadelphia 76ers':'PHI', 'Memphis Grizzlies':'MEM','Dallas Mavericks':'DAL','Houston Rockets':'HOU','San Antonio Spurs':'SAS','Utah Jazz':'UTA', 'Milwaukee Bucks':'MIL', 'Washington Wizards':'WAS'  }, inplace=True)

In [24]:
players.Tm.replace({'WSB':'WAS','NJN':'BRK', 'SEA': 'OKC', 'CHH':'CHA', 'VAN':'MEM', 'NOH':'NOP', 'CHA':'NOP','NOK':'NOP','CHO':'NOP'},inplace = True)

In [25]:
players.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1,Alaa Abdelnaby,PF,22,POR,43,0,6.7,1.3,2.7,...,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,1991
1,2,Mahmoud Abdul-Rauf,PG,21,DEN,67,19,22.5,6.2,15.1,...,0.5,1.3,1.8,3.1,0.8,0.1,1.6,2.2,14.1,1991
2,3,Mark Acres,C,28,ORL,68,0,19.3,1.6,3.1,...,2.1,3.2,5.3,0.4,0.4,0.4,0.6,3.2,4.2,1991
3,4,Michael Adams,PG,28,DEN,66,66,35.5,8.5,21.5,...,0.9,3,3.9,10.5,2.2,0.1,3.6,2.5,26.5,1991
4,5,Mark Aguirre,SF,31,DET,78,13,25.7,5.4,11.7,...,1.7,3.1,4.8,1.8,0.6,0.3,1.6,2.7,14.2,1991


In [26]:
df.head()

,State_Name,NBA_Teams
0,Arizona,PHO
1,California,GSW
1,California,LAC
1,California,LAL
1,California,SAC


In [27]:
driver.get('https://www.vacationcluster.com/trip-ideas/americas-20-best-cities-for-nightlife/')

In [28]:
estados=driver.find_elements_by_tag_name('h3')


In [29]:
states = []
for i in estados:
    states.append(i.text)

In [30]:
states[0]

'1. Miami Beach, Florida'

In [31]:
states1 = []
for i in states:
    states1.append(i.split(',')[1])
    

In [32]:
import re

In [33]:
states2 = []
for i in states1:
    states2.append(re.sub('^[\s]','', i))

In [34]:
estados = pd.DataFrame(states2)

In [35]:
estados.value_counts()

California       3
Florida          3
Tennessee        2
Texas            2
Colorado         1
Georgia          1
Illinois         1
Louisiana        1
Massachusetts    1
Nevada           1
New York         1
Oregon           1
Rhode Island     1
Washington       1
dtype: int64

In [36]:
df.head()

,State_Name,NBA_Teams
0,Arizona,PHO
1,California,GSW
1,California,LAC
1,California,LAL
1,California,SAC


In [37]:
top_states = ['California','Florida','Tennesse','Texas','New York']

In [38]:
players2 = players.set_index('Tm').join(df.set_index('NBA_Teams'), lsuffix='', rsuffix='_dcha', how='left')

In [39]:
players2 = players2.reset_index()
players2.rename(columns = {'index':'Team'}, inplace = True)

In [40]:
players2.head()

,Team,Rk,Player,Pos,Age,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,State_Name
0,ATL,23,John Battle,SG,28,79,2,23.6,5,10.9,...,1.6,2,2.7,0.6,0.1,1.4,1.8,13.6,1991,Georgia
1,ATL,110,Duane Ferrell,SF,25,78,2,14.9,2.2,4.6,...,1.1,2.3,0.7,0.4,0.3,1,1.9,6.1,1991,Georgia
2,ATL,188,Jon Koncak,C,27,77,61,25.1,1.8,4.2,...,3.6,4.9,1.6,1,1,0.6,3.4,4.1,1991,Georgia
3,ATL,196,Gary Leonard,C,23,4,0,2.3,0,0,...,0.5,0.5,0,0,0.3,0,0.5,0.5,1991,Georgia
4,ATL,213,Moses Malone*,C,35,82,15,23.3,3.4,7.3,...,4.8,8.1,0.8,0.4,0.9,1.7,1.6,10.6,1991,Georgia


### Cruce de tablas en función de la vida nocturna y ciudades con menos impuestos

In [41]:
def jaleito(row):
    if row['State_Name'] in top_states:
        
        val = True
    else:
        val = False
        
    return val

In [42]:
players2['Jaleito'] = players2.apply(jaleito, axis=1)

In [44]:
states_free = ['Tennessee','Florida','Texas','Washington DC*','Pennsylvania', 'Indiana']

In [46]:
def racano(row):
    if row['State_Name'] in states_free:
        
        val = True
    else:
        val = False
        
    return val

In [47]:
players2['Dinerito'] = players2.apply(racano, axis=1)

In [48]:
players2.Player = players2.Player.replace('*','')

In [49]:
players2

,Team,Rk,Player,Pos,Age,G,GS,MP,FG,FGA,...,AST,STL,BLK,TOV,PF,PTS,Year,State_Name,Jaleito,Dinerito
0,ATL,23,John Battle,SG,28,79,2,23.6,5,10.9,...,2.7,0.6,0.1,1.4,1.8,13.6,1991,Georgia,False,False
1,ATL,110,Duane Ferrell,SF,25,78,2,14.9,2.2,4.6,...,0.7,0.4,0.3,1,1.9,6.1,1991,Georgia,False,False
2,ATL,188,Jon Koncak,C,27,77,61,25.1,1.8,4.2,...,1.6,1,1,0.6,3.4,4.1,1991,Georgia,False,False
3,ATL,196,Gary Leonard,C,23,4,0,2.3,0,0,...,0,0,0.3,0,0.5,0.5,1991,Georgia,False,False
4,ATL,213,Moses Malone*,C,35,82,15,23.3,3.4,7.3,...,0.8,0.4,0.9,1.7,1.6,10.6,1991,Georgia,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18039,WAS,434,Jerome Robinson,SG,23,17,6,17.9,1.6,5.6,...,1.5,0.7,0.4,1.1,1.8,4.9,2021,Washington DC*,False,True
18040,WAS,463,Ish Smith,PG,32,44,1,21,2.9,6.8,...,3.9,0.7,0.3,0.9,1.4,6.7,2021,Washington DC*,False,True
18041,WAS,506,Moritz Wagner,C,23,25,13,15,2.5,4.9,...,1.3,0.9,0.3,0.8,2.5,7.1,2021,Washington DC*,False,True
18042,WAS,517,Russell Westbrook,PG,32,65,65,36.4,8.4,19,...,11.7,1.4,0.4,4.8,2.9,22.2,2021,Washington DC*,False,True


In [50]:
players2.to_csv('../players.csv')

In [51]:
players2.tail()

,Team,Rk,Player,Pos,Age,G,GS,MP,FG,FGA,...,AST,STL,BLK,TOV,PF,PTS,Year,State_Name,Jaleito,Dinerito
18039,WAS,434,Jerome Robinson,SG,23,17,6,17.9,1.6,5.6,...,1.5,0.7,0.4,1.1,1.8,4.9,2021,Washington DC*,False,True
18040,WAS,463,Ish Smith,PG,32,44,1,21,2.9,6.8,...,3.9,0.7,0.3,0.9,1.4,6.7,2021,Washington DC*,False,True
18041,WAS,506,Moritz Wagner,C,23,25,13,15,2.5,4.9,...,1.3,0.9,0.3,0.8,2.5,7.1,2021,Washington DC*,False,True
18042,WAS,517,Russell Westbrook,PG,32,65,65,36.4,8.4,19,...,11.7,1.4,0.4,4.8,2.9,22.2,2021,Washington DC*,False,True
18043,WAS,532,Cassius Winston,PG,22,22,0,4.5,0.6,1.5,...,0.5,0.1,0,0.2,0.4,1.9,2021,Washington DC*,False,True


In [52]:
players2 = players2[['Team','Player','Age','FG%','3P%','2P%','eFG%','FT%','PTS','Year','State_Name','Jaleito','Dinerito']]

In [53]:
players2 = players2.drop(players2[(players2.Player == 'Player')].index)


In [54]:
players2[["Age", 'FG%','3P%','2P%','eFG%','FT%','PTS']] = players2[["Age", 'FG%','3P%','2P%','eFG%','FT%','PTS']].apply(pd.to_numeric)

In [55]:
players2.to_csv('../players.csv')

In [92]:
players2.astype({"Jaleito": str})
players2.astype({"Dinerito": str})

,Team,Player,Age,FG,3P,2P,eFG,FT,PTS,Year,State_Name,Jaleito,Dinerito
0,ATL,John Battle,28,0.461,0.286,0.471,0.469,0.854,13.6,1991,Georgia,False,False
1,ATL,Duane Ferrell,25,0.489,0.667,0.487,0.492,0.801,6.1,1991,Georgia,False,False
2,ATL,Jon Koncak,27,0.436,0.125,0.444,0.438,0.593,4.1,1991,Georgia,False,False
3,ATL,Gary Leonard,23,0.000,0.000,0.000,0.000,0.500,0.5,1991,Georgia,False,False
4,ATL,Moses Malone*,35,0.468,0.000,0.474,0.468,0.831,10.6,1991,Georgia,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18039,WAS,Jerome Robinson,23,0.295,0.262,0.321,0.353,0.800,4.9,2021,Washington DC*,False,True
18040,WAS,Ish Smith,32,0.434,0.367,0.448,0.465,0.576,6.7,2021,Washington DC*,False,True
18041,WAS,Moritz Wagner,23,0.508,0.310,0.613,0.561,0.788,7.1,2021,Washington DC*,False,True
18042,WAS,Russell Westbrook,32,0.439,0.315,0.475,0.474,0.656,22.2,2021,Washington DC*,False,True


In [101]:
import mysql.connector as conn

In [102]:
db=conn.connect(host='localhost', user='root', passwd='admin', database='nba')

cursor=db.cursor()

In [103]:
cursor.execute('drop table if exists players;')



tabla='''

        create table players (
            
            Team varchar(50),
            Player varchar(50),
            Age int,
            FG float,
            3P float,
            2P float,
            eFG float,
            FT float,
            PTS float,
            Year int,
            State_Name varchar(50),
            Jaleito varchar(50),
            Dinerito varchar(50)
            
        );

'''


cursor.execute(tabla)

In [104]:
players2.rename(columns = {'FG%':'FG', '3P%':'3P','2P%': '2P','eFG%':'eFG','FT%':'FT'}, inplace = True)

In [105]:
players2.State_Name.fillna('unknown')

0               Georgia
1               Georgia
2               Georgia
3               Georgia
4               Georgia
              ...      
18039    Washington DC*
18040    Washington DC*
18041    Washington DC*
18042    Washington DC*
18043    Washington DC*
Name: State_Name, Length: 17386, dtype: object

In [106]:
players2.columns

Index(['Team', 'Player', 'Age', 'FG', '3P', '2P', 'eFG', 'FT', 'PTS', 'Year',
       'State_Name', 'Jaleito', 'Dinerito'],
      dtype='object')

In [107]:
players2.State_Name.fillna('uknown',inplace=True)

In [108]:
for i in range(len(players2)):
    
    insert_query='insert into {} ({}) values {};'\
                 .format('players', ','.join(players2.columns), tuple(players2.iloc[i].values))
    
    
   
    
    
    cursor.execute(insert_query)
    
    
db.commit()